In [26]:
%matplotlib inline

# Import a bunch of libraries.
import re
import numpy as np
import pandas as pd
import time

In [27]:
debug_wrangling = False # if True, uses existing mini_initial.csv file so this runs fast
save_data = True # if false, will not save, which takes the most time


In [28]:
def load_data(filename):
    
    dtypes = {
        'MachineIdentifier':                                    'str',
        'ProductName':                                          'str',
        'EngineVersion':                                        'str',
        'AppVersion':                                           'str',
        'AvSigVersion':                                         'str',
        'IsBeta':                                               'int8',
        'RtpStateBitfield':                                     'float64',
        'IsSxsPassiveMode':                                     'int8',
        'DefaultBrowsersIdentifier':                            'float32',
        'AVProductStatesIdentifier':                            'float32',
        'AVProductsInstalled':                                  'float16',
        'AVProductsEnabled':                                    'float16',
        'HasTpm':                                               'int8',
        'CountryIdentifier':                                    'int16',
        'CityIdentifier':                                       'float32',
        'OrganizationIdentifier':                               'float16',
        'GeoNameIdentifier':                                    'float16',
        'LocaleEnglishNameIdentifier':                          'int16',
        'Platform':                                             'category',
        'Processor':                                            'category',
        'OsVer':                                                'category',
        'OsBuild':                                              'int16',
        'OsSuite':                                              'int16',
        'OsPlatformSubRelease':                                 'category',
        'OsBuildLab':                                           'category',
        'SkuEdition':                                           'category',
        'IsProtected':                                          'float16',
        'AutoSampleOptIn':                                      'int8',
        'PuaMode':                                              'category',
        'SMode':                                                'float16',
        'IeVerIdentifier':                                      'float16',
        'SmartScreen':                                          'str',
        'Firewall':                                             'float16',
        'UacLuaenable':                                         'float64', 
        'Census_MDC2FormFactor':                                'category',
        'Census_DeviceFamily':                                  'category',
        'Census_OEMNameIdentifier':                             'float32', 
        'Census_OEMModelIdentifier':                            'float32',
        'Census_ProcessorCoreCount':                            'float16',
        'Census_ProcessorManufacturerIdentifier':               'float16',
        'Census_ProcessorModelIdentifier':                      'float32', 
        'Census_ProcessorClass':                                'category',
        'Census_PrimaryDiskTotalCapacity':                      'float64', 
        'Census_PrimaryDiskTypeName':                           'category',
        'Census_SystemVolumeTotalCapacity':                     'float64', 
        'Census_HasOpticalDiskDrive':                           'int8',
        'Census_TotalPhysicalRAM':                              'float32',
        'Census_ChassisTypeName':                               'str',
        'Census_InternalPrimaryDiagonalDisplaySizeInInches':    'float32', 
        'Census_InternalPrimaryDisplayResolutionHorizontal':    'float32', 
        'Census_InternalPrimaryDisplayResolutionVertical':      'float32', 
        'Census_PowerPlatformRoleName':                         'category',
        'Census_InternalBatteryType':                           'str',
        'Census_InternalBatteryNumberOfCharges':                'float64', 
        'Census_OSVersion':                                     'category',
        'Census_OSArchitecture':                                'category',
        'Census_OSBranch':                                      'category',
        'Census_OSBuildNumber':                                 'int16',
        'Census_OSBuildRevision':                               'int32',
        'Census_OSEdition':                                     'str',
        'Census_OSSkuName':                                     'category',
        'Census_OSInstallTypeName':                             'category',
        'Census_OSInstallLanguageIdentifier':                   'float16',
        'Census_OSUILocaleIdentifier':                          'int16',
        'Census_OSWUAutoUpdateOptionsName':                     'category',
        'Census_IsPortableOperatingSystem':                     'int8',
        'Census_GenuineStateName':                              'category',
        'Census_ActivationChannel':                             'category',
        'Census_IsFlightingInternal':                           'float16',
        'Census_IsFlightsDisabled':                             'float16',
        'Census_FlightRing':                                    'category',
        'Census_ThresholdOptIn':                                'float16',
        'Census_FirmwareManufacturerIdentifier':                'float16',
        'Census_FirmwareVersionIdentifier':                     'float32',
        'Census_IsSecureBootEnabled':                           'int8',
        'Census_IsWIMBootEnabled':                              'float16',
        'Census_IsVirtualDevice':                               'float16',
        'Census_IsTouchEnabled':                                'int8',
        'Census_IsPenCapable':                                  'int8',
        'Census_IsAlwaysOnAlwaysConnectedCapable':              'float16',
        'Wdft_IsGamer':                                         'float16',
        'Wdft_RegionIdentifier':                                'float16',
        'HasDetections':                                        'int8'
        }

    df = pd.read_csv(filename, dtype=dtypes, engine='c')
    return df

In [29]:
def clean_data(df):
        
    #
    # make all strings lower case
    # get rid of hex char codes, keep the actual code number
    #
    
    char_treatment = [
        'AvSigVersion',
        'SmartScreen',
        'Census_InternalBatteryType'
    ]
    
    case_treatment = [
        'SmartScreen',
        'Census_ChassisTypeName',
        'Census_OSEdition',
        'Census_PowerPlatformRoleName',
        'Census_GenuineStateName'
    ]
    
    print("-- replacing weird characters ...")
    for col in char_treatment:
        if df[col].dtype.name == 'object':
            df[col] = df[col].str.replace(r'&#x(\d\d);', '\1', regex=True)
            df[col] = df[col].str.replace(r'[\x00-\x1f]', '', regex=True)
            
    print("-- lower-casing where appropriate ...")
    for col in case_treatment:
        if df[col].dtype.name == 'object':
            df[col] = df[col].str.lower()
            
    #
    # make strings into categories
    #
    
    categories = [
        'SmartScreen',
        'Census_InternalBatteryType',
        'Census_ChassisTypeName',
        'Census_OSEdition'
    ]
    
    print("-- making categories from strings that needed massaging ...")
    for col in categories:
        df[col] = df[col].astype('category')


        
    #
    # add 'unknown' categories where necessary and replace the NAs
    # ADD COLUMNS NAMES HERE TO HAVE THEIR CATEGORIES AUGMENTED AND NAS FILLED WITH 'unknown'
    #
    
    categories = [
        'PuaMode',
        'SmartScreen',
        'Census_ProcessorClass',
        'Census_PrimaryDiskTypeName',  # ['HDD' 'SSD' 'UNKNOWN' 'Unspecified']
        'Census_InternalBatteryType',
        'Census_OSEdition',
        'Census_PowerPlatformRoleName', # also had 'unknown' as well as Nas
        'Census_GenuineStateName'       # and this one too
        
    ]

    print("-- adding categories ..")
    for col in categories:
        df[col].cat.add_categories(['unknown'], inplace=True)
        df[col].fillna('unknown', inplace=True)
    # add one manually because it needs a special unknown value
    df["OsBuildLab"].cat.add_categories(["0.0.-.-.0-0"], inplace=True)
    df["OsBuildLab"].fillna("0.0.-.-.0-0", inplace=True)
    # and this one already had some 'unknown' values
    df['Census_ChassisTypeName'].fillna('unknown', inplace=True)



    #
    # flag and fill selected NAs
    # ADD COLUMN NAMES HERE IN nafill TO HAVE COLUMNS FLAGGED AND FILLED WITH PROVIDED VALUES
    #
    
    print("-- replacing selected NA values")
    nafill = {
        "RtpStateBitfield":0,
        "DefaultBrowsersIdentifier":0,
        "AVProductStatesIdentifier":0,
        "AVProductsInstalled":0,
        "AVProductsEnabled":0,
        "CityIdentifier":0,
        "OrganizationIdentifier":0,
        "GeoNameIdentifier":0,
        "IsProtected":0,
        "SMode":0,
        "IeVerIdentifier":0,
        "Firewall":0,
        "UacLuaenable":0,
        "Census_OEMNameIdentifier":0,
        "Census_OEMModelIdentifier":0,
        "Census_ProcessorCoreCount":0,
        "Census_ProcessorManufacturerIdentifier":0,
        "Census_ProcessorModelIdentifier":0,
        "Census_PrimaryDiskTotalCapacity":0,
        "Census_SystemVolumeTotalCapacity":0,
        "Census_TotalPhysicalRAM":0,
        "Census_InternalPrimaryDiagonalDisplaySizeInInches":0,
        "Census_InternalPrimaryDisplayResolutionHorizontal":0,
        "Census_InternalPrimaryDisplayResolutionVertical":0,
        "Census_InternalBatteryNumberOfCharges":0,
        "Census_OSInstallLanguageIdentifier":0,
        "Census_IsFlightingInternal":0,
        "Census_IsFlightsDisabled":0,
        "Census_ThresholdOptIn":0,
        "Census_FirmwareManufacturerIdentifier":0,
        "Census_IsWIMBootEnabled":0,
        "Census_IsVirtualDevice":0,
        "Census_IsAlwaysOnAlwaysConnectedCapable":0,
        "Wdft_IsGamer":0,
        "Wdft_RegionIdentifier":0
        
        
    }

    for col in nafill:
        df[col+'_wasna'] = df[col].isna()
    df.fillna(value=nafill, inplace=True)
    
    #
    # then some of these columns can become ints, not floats
    #

    print("-- converting columns to int ...")
    df['RtpStateBitfield'] = df['RtpStateBitfield'].astype(np.uint8)

    #
    # deal with version numbers
    #
    
    def map_version2(df,col):
        df_split = df[col].str.split(".", n=3, expand=True)
        df[col+'_major'] = df_split[0].astype(np.int16)
        df[col+'_minor'] = df_split[1].astype(np.int16)
        df[col+'_build1'] = df_split[2].astype(np.int16)
        df[col+'_build2'] = df_split[3].astype(np.int16)
        # the "combined" column is an attempt at making an orginal out of the four values
        df[col+'_combined'] = 10000.0*df[col+'_major']+df[col+'_minor']+(df[col+'_build1'])/1000.0+(df[col+'_build2'])/1000000.0

    print("-- mapping version numbers ...")
    map_version2(df, "EngineVersion")
    map_version2(df, "AppVersion")
    map_version2(df, "AvSigVersion")
    map_version2(df, "Census_OSVersion")
    map_version2(df, "OsVer")

    #
    # this one is more complicated:
    # OsBuildLab: example: 7134.1.amd64fre.rs4_release.180410-1804
    #

    print("-- mapping OsBuildLab version numbers ...")
    # one stupid row in the test set has an asterisk instead of a dot
    series = df['OsBuildLab'].str.replace('*', '.', regex=False)
    # now parse
    df_split = series.str.split(".", n=4, expand=True)
    df['OsBuildLab_major'] = df_split[0].astype(np.int16)
    df['OsBuildLab_minor'] = df_split[1].astype(np.int16)
    df['OsBuildLab_platform'] = df_split[2].astype('category')
    df['OsBuildLab_release'] = df_split[3].astype('category')
    df_build = df_split[4].str.split("-", n=1, expand=True)
    df['OsBuildLab_build1'] = df_build[0].astype(np.int32)
    df['OsBuildLab_build2'] = df_build[1].astype(np.int32)
    # the "combined" column is an attempt at making an orginal out of the four values
    df['OsBuildLab_combined'] = 10000.0*df['OsBuildLab_major']+df['OsBuildLab_minor']+df['OsBuildLab_build1']/1000.0+df['OsBuildLab_build2']/1000000.0
    df_split = None
    df_build = None
    
    return df


In [30]:
# function to create new mini train, test and dev samples 
# pulling data from the training set using random stratification on the outcome variable

from sklearn.model_selection import train_test_split
def generate_train_test_dev_minis(n, features, labels):
    sample_size = n / features.shape[0]
    reserved_size = 1-sample_size
    X_train, X_test_and_dev_and_rest, y_train, y_test_and_dev_and_rest = train_test_split(features, labels, stratify=labels, test_size=reserved_size, random_state=0)
    reserved_size = 1-X_train.shape[0]/X_test_and_dev_and_rest.shape[0]
    X_test, X_dev_and_rest, y_test, y_dev_and_rest = train_test_split(X_test_and_dev_and_rest, y_test_and_dev_and_rest, stratify=y_test_and_dev_and_rest, test_size=reserved_size, random_state=0)
    reserved_size = 1-X_test.shape[0]/X_dev_and_rest.shape[0]
    X_dev, X_rest, y_dev, y_rest = train_test_split(X_dev_and_rest, y_dev_and_rest, stratify=y_dev_and_rest, test_size=reserved_size, random_state=0)
    return X_train, X_test, X_dev, y_train, y_test, y_dev


In [31]:
# improved function to create a mini set from the supplied criteria (n, features, labels)
from sklearn.model_selection import train_test_split
def generate_mini(n, features, labels):
    sample_size = n / features.shape[0]
    reserved_size = 1-sample_size
    X_mini, X_rest, y_mini, y_rest = train_test_split(features, labels, stratify=labels, test_size=reserved_size, random_state=0)
    return X_mini, X_rest, y_mini, y_rest

In [32]:
# function to create stratified train, dev and test sets from supplied ratios 
from sklearn.model_selection import train_test_split
def generate_train_test_dev_sets(train_ratio, test_ratio, features, labels):
    reserved_size = 1-train_ratio
    X_train, X_test_and_dev, y_train, y_test_and_dev = train_test_split(features, labels, stratify=labels, test_size=reserved_size, random_state=0)
    reserved_size = 1 - (test_ratio / reserved_size)
    X_test, X_dev, y_test, y_dev = train_test_split(X_test_and_dev, y_test_and_dev, stratify=y_test_and_dev, test_size=reserved_size, random_state=0)
    return X_train, X_test, X_dev, y_train, y_test, y_dev

In [33]:
#
# function for saving frames, also dropping machine ids
#

def save_files(df, name):
    
    # drop machine identifiers, so models don't train on them
    df = df.drop(columns=["MachineIdentifier"])
        
    # save the file
    df.to_csv("data/"+name+"_clean.csv", index=False)
    
    

In [34]:
#
# main code part one: train dataset
#

start = time.time()

if debug_wrangling:
    print("using mini_initial.csv dataset")
    filename = "data/mini_initial.csv"
else:        
    print("using big dataset")
    filename = "data/train.csv"
    
#
# initial load and casting to desired types
#

df = load_data(filename)
print("done loading train data")
print("total rows in set:", len(df))
print("seconds elapsed:", time.time()-start)

#
# make mini_initial, before cleaning and dropping, if necessary (to debug wrangler code)
#
   
if not debug_wrangling:
    print("creating mini_initial.csv for debug purposes ...")
    df2 = df.sample(100000, random_state=123)
    df2.to_csv("data/mini_initial.csv", index=False)
    print("created mini_initial.csv data")
    print("seconds elapsed:", time.time()-start)
    
#
# main work is done here
#
    
print("cleaning data ...")
df = clean_data(df)
print("done cleaning train data")
print("seconds elapsed:", time.time()-start)

if save_data:
    print("saving data files ...")
    
    # generate stratified data sets using supplied ratios to create the sets
    df_train, df_test, df_dev, train_labels, test_labels, dev_labels = \
        generate_train_test_dev_sets(.7, .15, df, df['HasDetections'])
    
    df=[] # release df from memory
    
    # Changing out the below for the above
    # also split out a big dev and test file here, let's say 1,000,000
    #df_dev = df.sample(1000000 if not debug_wrangling else 500, random_state=123).copy()
    #df = df.drop(df_dev.index)

    #df_test = df.sample(1000000 if not debug_wrangling else 500, random_state=123).copy()
    #df = df.drop(df_test.index)

    #
    # save clean master file
    #

    print("saving train_clean ...")
    save_files(df_train, "train")
    print("done saving clean train file")

    print("saving test_clean ...")
    save_files(df_test, "test")
    print("done saving clean test file")

    print("saving dev_clean ...")
    save_files(df_dev, "dev")
    print("done saving clean dev file")

        # make this work with a smaller size for the mini_initial set
    sample_size = 50000 if len(df_train) > 1000000 else 500

    # now call Kevin's code for the work
    mini_ratio = .05

    mini_train, remaining, mini_train_labels, remaining_labels = \
        generate_mini(df_train.shape[0]*mini_ratio, df_train, df_train['HasDetections'])

    mini_dev, remaining, mini_dev_labels, remaining_labels = \
        generate_mini(df_dev.shape[0]*mini_ratio, df_dev, df_dev['HasDetections'])

    mini_test, remaining, mini_test_labels, remaining_labels = \
        generate_mini(df_test.shape[0]*mini_ratio, df_test, df_test['HasDetections'])
    
    print("shape of mini_train:",mini_train.shape)
    print("shape of mini_dev:",mini_dev.shape) 
    print("shape of mini_test:",mini_test.shape)
    print("done making mini sets")

    #
    # save clean mini files, all with labels
    #

    print("saving mini_train_clean ...")
    save_files(mini_train, "mini_train")
    print("saving mini_dev_clean ...")
    save_files(mini_dev, "mini_dev")
    print("saving mini_test_clean ...")
    save_files(mini_test, "mini_test")
    print("done saving mini files")
    
    print("seconds elapsed:", time.time()-start)
    
if not debug_wrangling:
    print("loading test data ...")
    df_test = load_data("data/test.csv")
    print("done loading test data")
    print("seconds elapsed:", time.time()-start)
    print("total rows in test set:", len(df_test))
    print("cleaning test data ...")
    clean_data(df_test)
    print("done cleaning test data")
    print("seconds elapsed:", time.time()-start)
    if save_data:
        print("saving clean test data ...")
        save_files(df_test, "real_test")
        print("done saving clean files")


print("seconds elapsed:", time.time()-start)



using big dataset
done loading train data
total rows in set: 8921483
seconds elapsed: 297.5023922920227
creating mini_initial.csv for debug purposes ...
created mini_initial.csv data
seconds elapsed: 311.84639620780945
cleaning data ...
-- replacing weird characters ...
-- lower-casing where appropriate ...
-- making categories from strings that needed massaging ...
-- adding categories ..
-- replacing selected NA values
-- converting columns to int ...
-- mapping version numbers ...
-- mapping OsBuildLab version numbers ...
done cleaning train data
seconds elapsed: 550.6064851284027
saving data files ...
saving train_clean ...
done saving clean train file
saving test_clean ...
done saving clean test file
saving dev_clean ...
done saving clean dev file
shape of mini_train: (312251, 150)
shape of mini_dev: (66911, 150)
shape of mini_test: (66911, 150)
done making mini sets
saving mini_train_clean ...
saving mini_dev_clean ...
saving mini_test_clean ...
done saving mini files
seconds ela

In [40]:
for dtype in df_test.dtypes.items():
    print("'{:} '{:}',".format((dtype[0] + "':").ljust(54), dtype[1]))

'MachineIdentifier':                                    'object',
'ProductName':                                          'object',
'EngineVersion':                                        'object',
'AppVersion':                                           'object',
'AvSigVersion':                                         'object',
'IsBeta':                                               'int8',
'RtpStateBitfield':                                     'uint8',
'IsSxsPassiveMode':                                     'int8',
'DefaultBrowsersIdentifier':                            'float32',
'AVProductStatesIdentifier':                            'float32',
'AVProductsInstalled':                                  'float16',
'AVProductsEnabled':                                    'float16',
'HasTpm':                                               'int8',
'CountryIdentifier':                                    'int16',
'CityIdentifier':                                       'float32',
'Organization

In [47]:
# Unit testing the map_version2 routine
unitTestData = {'EngineVersion':['1.1.12902.0', '1.1.13000.0', '1.1.13103.0', '1.1.13202.0', '1.1.13303.0',
                                 '1.1.13407.0', '1.1.13504.0', '1.1.13601.0', '1.1.13701.0', '1.1.13804.0',
                                 '1.1.13903.0', '1.1.14003.0', '1.1.14104.0', '1.1.14202.0', '1.1.14303.0',
                                 '1.1.14305.0', '1.1.14306.0', '1.1.14405.2', '1.1.14500.5', '1.1.14600.4',
                                 '1.1.14700.5', '1.1.14800.1', '1.1.14800.3', '1.1.14901.3', '1.1.14901.4',
                                 '1.1.15000.1', '1.1.15000.2', '1.1.15100.1', '1.1.15200.1', '1.1.15300.5',
                                 '1.1.15300.6', '1.1.12805.0', '1.1.13704.0', '1.1.14700.3', '1.1.14700.4',
                                 '1.1.12101.0', '1.1.13802.0', '1.1.11502.0', '1.1.11701.0', '1.1.14002.0',
                                 '1.1.14102.0', '1.1.14201.0', '1.1.14500.2', '1.1.13102.0', '1.1.12400.0',
                                 '1.1.12603.0', '1.1.14103.0', '1.1.13902.0', '1.1.14001.0', '1.1.11903.0'],
                   'AppVersion':['4.10.14393.0', '4.10.14393.1066', '4.10.14393.1198', '4.10.14393.1593',
                                 '4.10.14393.1613', '4.10.14393.1794', '4.10.14393.2273', '4.10.14393.953',
                                 '4.10.205.0', '4.10.209.0', '4.11.15063.0', '4.11.15063.1155',
                                 '4.11.15063.447', '4.12.16299.15', '4.12.17007.17123', '4.12.17007.18011',
                                 '4.12.17007.18022', '4.13.17134.1', '4.13.17134.112', '4.13.17134.191',
                                 '4.13.17134.228', '4.14.17613.18038', '4.14.17613.18039', '4.14.17639.18041',
                                 '4.16.17656.18052', '4.17.17686.1003', '4.18.1806.18062', '4.18.1807.18075',
                                 '4.18.1807.20063', '4.18.1809.2', '4.5.218.0', '4.6.305.0', '4.8.10240.16384',
                                 '4.8.10240.17071', '4.8.10240.17113', '4.8.10240.17202', '4.8.10240.17394',
                                 '4.8.10240.17443', '4.8.10240.17609', '4.8.10240.17861', '4.8.10240.17889',
                                 '4.8.10240.17914', '4.8.207.0', '4.9.10586.0', '4.9.10586.1045',
                                 '4.9.10586.1106', '4.9.10586.494', '4.9.10586.589', '4.9.10586.672',
                                 '4.9.10586.839'],
                 'AvSigVersion':['1.225.1338.0', '1.225.1947.0', '1.225.394.0', '1.227.2357.0', '1.227.2846.0',
                                 '1.227.2939.0', '1.229.1557.0', '1.229.1669.0', '1.229.1848.0',
                                 '1.231.1226.0', '1.231.1362.0', '1.233.1808.0', '1.235.1725.0',
                                 '1.235.2115.0', '1.235.2586.0', '1.235.428.0', '1.237.0.0', '1.237.1259.0',
                                 '1.237.1572.0', '1.237.1702.0', '1.237.782.0', '1.237.787.0', '1.239.1512.0',
                                 '1.239.230.0', '1.239.37.0', '1.239.460.0', '1.239.956.0', '1.241.296.0',
                                 '1.241.643.0', '1.245.1013.0', '1.245.1110.0', '1.245.449.0', '1.245.931.0',
                                 '1.245.977.0', '1.247.347.0', '1.247.641.0', '1.249.1361.0', '1.249.281.0',
                                 '1.249.557.0', '1.249.713.0', '1.249.894.0', '1.251.1027.0', '1.251.1500.0',
                                 '1.251.170.0', '1.251.359.0', '1.251.42.0', '1.251.505.0', '1.251.878.0',
                                 '1.253.1015.0', '1.253.138.0'],
             'Census_OSVersion':['10.0.10240.16384', '10.0.10240.16397', '10.0.10240.16405',
                                 '10.0.10240.16445', '10.0.10240.16463', '10.0.10240.16487',
                                 '10.0.10240.16644', '10.0.10240.17071', '10.0.10240.17202',
                                 '10.0.10240.17394', '10.0.10240.17443', '10.0.10240.17709',
                                 '10.0.10240.17861', '10.0.10240.17889', '10.0.10240.17914', '10.0.10586.0',
                                 '10.0.10586.1007', '10.0.10586.104', '10.0.10586.1045', '10.0.10586.1106',
                                 '10.0.10586.1176', '10.0.10586.122', '10.0.10586.14', '10.0.10586.164',
                                 '10.0.10586.17', '10.0.10586.218', '10.0.10586.29', '10.0.10586.3',
                                 '10.0.10586.318', '10.0.10586.36', '10.0.10586.420', '10.0.10586.494',
                                 '10.0.10586.545', '10.0.10586.589', '10.0.10586.63', '10.0.10586.633',
                                 '10.0.10586.679', '10.0.10586.71', '10.0.10586.713', '10.0.10586.753',
                                 '10.0.10586.839', '10.0.10586.873', '10.0.10586.916', '10.0.10586.962',
                                 '10.0.14393.0', '10.0.14393.105', '10.0.14393.1066', '10.0.14393.1198',
                                 '10.0.14393.1358', '10.0.14393.1480'],
                        'OsVer':['10.0.0.0', '10.0.0.1', '10.0.0.112', '10.0.0.2', '10.0.0.22', '10.0.0.250',
                                 '10.0.0.3', '10.0.0.80', '10.0.0.96', '10.0.1.0', '10.0.1.144', '10.0.1.244',
                                 '10.0.1.44', '10.0.153.153', '10.0.16.0', '10.0.16.36', '10.0.19.80',
                                 '10.0.2.0', '10.0.2.80', '10.0.2.86', '10.0.21.0', '10.0.23.0', '10.0.26.128',
                                 '10.0.3.0', '10.0.3.232', '10.0.3.80', '10.0.32.0', '10.0.32.72', '10.0.4.0',
                                 '10.0.4.80', '10.0.48.0', '10.0.5.0', '10.0.5.117', '10.0.5.18', '10.0.6.0',
                                 '10.0.64.150', '10.0.7.0', '10.0.7.101', '10.0.7.80', '10.0.72.0', '10.0.8.0',
                                 '10.0.80.0', '6.1.0.0', '6.1.0.112', '6.1.0.128', '6.1.1.0', '6.1.16.36',
                                 '6.1.2.0', '6.1.3.0', '6.1.4.0'],
                   'OsBuildLab':['10240.16384.amd64fre.th1.150709-1700',
                                 '10240.16393.amd64fre.th1_st1.150717-1719',
                                 '10240.16393.x86fre.th1_st1.150717-1719',
                                 '10240.16431.amd64fre.th1.150810-2333',
                                 '10240.16431.x86fre.th1.150810-2333',
                                 '10240.16463.amd64fre.th1.150819-1946',
                                 '10240.17071.amd64fre.th1.160802-1852',
                                 '10240.17113.amd64fre.th1.160906-1755',
                                 '10240.17202.amd64fre.th1_st1.161118-1836',
                                 '10240.17394.amd64fre.th1_st1.170427-1347',
                                 '10240.17443.amd64fre.th1.170602-2340',
                                 '10240.17443.x86fre.th1.170602-2340', '10240.17709.x86fre.th1.171130-0900',
                                 '10240.17861.amd64fre.th1.180427-1806',
                                 '10240.17889.amd64fre.th1_st1.180529-1823',
                                 '10240.17914.amd64fre.th1.180627-1911',
                                 '10586.0.amd64fre.th2_release.151029-1700',
                                 '10586.0.x86fre.th2_release.151029-1700',
                                 '10586.1007.amd64fre.th2_release.170706-2002',
                                 '10586.103.amd64fre.th2_release.160126-1819',
                                 '10586.1045.amd64fre.th2_release.170728-1941',
                                 '10586.1045.x86fre.th2_release.170728-1941',
                                 '10586.11.amd64fre.th2_release.151112-1900',
                                 '10586.1106.amd64fre.th2_release.170904-1742',
                                 '10586.1176.amd64fre.th2_release_sec.170913-1848',
                                 '10586.1176.x86fre.th2_release_sec.170913-1848',
                                 '10586.122.amd64fre.th2_release_inmarket.160222-1549',
                                 '10586.162.amd64fre.th2_release_sec.160223-1728',
                                 '10586.162.x86fre.th2_release_sec.160223-1728',
                                 '10586.17.amd64fre.th2_release.151121-2308',
                                 '10586.212.amd64fre.th2_release_sec.160328-1908',
                                 '10586.212.x86fre.th2_release_sec.160328-1908',
                                 '10586.3.amd64fre.th2_release_sec.151104-1948',
                                 '10586.306.amd64fre.th2_release_sec.160422-1850',
                                 '10586.420.amd64fre.th2_release_sec.160527-1834',
                                 '10586.420.x86fre.th2_release_sec.160527-1834',
                                 '10586.494.amd64fre.th2_release_sec.160630-1736',
                                 '10586.494.x86fre.th2_release_sec.160630-1736',
                                 '10586.545.amd64fre.th2_release.160802-1857',
                                 '10586.589.amd64fre.th2_release.160906-1759',
                                 '10586.63.amd64fre.th2_release.160104-1513',
                                 '10586.633.amd64fre.th2_release.161004-1602',
                                 '10586.672.amd64fre.th2_release_sec.161024-1825',
                                 '10586.672.x86fre.th2_release_sec.161024-1825',
                                 '10586.839.amd64fre.th2_release.170303-1605',
                                 '10586.839.x86fre.th2_release.170303-1605',
                                 '10586.916.amd64fre.th2_release_sec.170427-1350',
                                 '10586.962.amd64fre.th2_release.170602-2241',
                                 '10586.962.x86fre.th2_release.170602-2241',
                                 '14393.0.amd64fre.rs1_release.160715-1616']
               }

unitTestDf = pd.DataFrame(unitTestData) 
unitTestDf
priorDf = unitTestDf
# method to test (copied from the notebook)
def map_version2(df,col):
    df_split = df[col].str.split(".", n=3, expand=True)
    df[col+'_major'] = df_split[0].astype(np.int16)
    df[col+'_minor'] = df_split[1].astype(np.int16)
    df[col+'_build1'] = df_split[2].astype(np.int16)
    df[col+'_build2'] = df_split[3].astype(np.int16)
    # the "combined" column is an attempt at making an orginal out of the four values
    df[col+'_combined'] = 10000.0*df[col+'_major']+df[col+'_minor']+(df[col+'_build1'])/1000.0+(df[col+'_build2'])/1000000.0

map_version2(unitTestDf, "EngineVersion")
map_version2(unitTestDf, "AppVersion")
map_version2(unitTestDf, "AvSigVersion")
map_version2(unitTestDf, "Census_OSVersion")
map_version2(unitTestDf, "OsVer")
unitTestDf

,EngineVersion,AppVersion,AvSigVersion,Census_OSVersion,OsVer,OsBuildLab,EngineVersion_major,EngineVersion_minor,EngineVersion_build1,EngineVersion_build2,...,Census_OSVersion_major,Census_OSVersion_minor,Census_OSVersion_build1,Census_OSVersion_build2,Census_OSVersion_combined,OsVer_major,OsVer_minor,OsVer_build1,OsVer_build2,OsVer_combined
0,1.1.12902.0,4.10.14393.0,1.225.1338.0,10.0.10240.16384,10.0.0.0,10240.16384.amd64fre.th1.150709-1700,1,1,12902,0,...,10,0,10240,16384,100010.256384,10,0,0,0,100000.000000
1,1.1.13000.0,4.10.14393.1066,1.225.1947.0,10.0.10240.16397,10.0.0.1,10240.16393.amd64fre.th1_st1.150717-1719,1,1,13000,0,...,10,0,10240,16397,100010.256397,10,0,0,1,100000.000001
2,1.1.13103.0,4.10.14393.1198,1.225.394.0,10.0.10240.16405,10.0.0.112,10240.16393.x86fre.th1_st1.150717-1719,1,1,13103,0,...,10,0,10240,16405,100010.256405,10,0,0,112,100000.000112
3,1.1.13202.0,4.10.14393.1593,1.227.2357.0,10.0.10240.16445,10.0.0.2,10240.16431.amd64fre.th1.150810-2333,1,1,13202,0,...,10,0,10240,16445,100010.256445,10,0,0,2,100000.000002
4,1.1.13303.0,4.10.14393.1613,1.227.2846.0,10.0.10240.16463,10.0.0.22,10240.16431.x86fre.th1.150810-2333,1,1,13303,0,...,10,0,10240,16463,100010.256463,10,0,0,22,100000.000022
5,1.1.13407.0,4.10.14393.1794,1.227.2939.0,10.0.10240.16487,10.0.0.250,10240.16463.amd64fre.th1.150819-1946,1,1,13407,0,...,10,0,10240,16487,100010.256487,10,0,0,250,100000.000250
6,1.1.13504.0,4.10.14393.2273,1.229.1557.0,10.0.10240.16644,10.0.0.3,10240.17071.amd64fre.th1.160802-1852,1,1,13504,0,...,10,0,10240,16644,100010.256644,10,0,0,3,100000.000003
7,1.1.13601.0,4.10.14393.953,1.229.1669.0,10.0.10240.17071,10.0.0.80,10240.17113.amd64fre.th1.160906-1755,1,1,13601,0,...,10,0,10240,17071,100010.257071,10,0,0,80,100000.000080
8,1.1.13701.0,4.10.205.0,1.229.1848.0,10.0.10240.17202,10.0.0.96,10240.17202.amd64fre.th1_st1.161118-1836,1,1,13701,0,...,10,0,10240,17202,100010.257202,10,0,0,96,100000.000096
9,1.1.13804.0,4.10.209.0,1.231.1226.0,10.0.10240.17394,10.0.1.0,10240.17394.amd64fre.th1_st1.170427-1347,1,1,13804,0,...,10,0,10240,17394,100010.257394,10,0,1,0,100000.001000
